In [1]:
import pandas as pd
import numpy as np
import sys
# from torch.utils.data import DataLoader

from transformers import AutoTokenizer, TrainingArguments, Trainer

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
import torch

from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# user_resp = pd.read_csv('dataset/user_response.csv')
user_resp = pd.read_csv('dataset/user_resp.csv')

In [3]:
user_resp.shape

(4039474, 3)

In [6]:
user_resp = user_resp.dropna(subset = ['clean_text'])

In [7]:
user_resp.clean_text.isna().sum()

0

In [15]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base', use_fast=True)
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=11)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = AutoModelForSequenceClassification.from_pretrained("model/model_MFT_all", num_labels=11).to(device)

In [16]:
import torch

class setDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# data_encodings = tokenizer(user_resp.clean_text.tolist(), truncation=True, max_length=500, padding=True)
# dataset = setDataset(data_encodings, user_resp.label_id)

In [17]:
def get_predictions(X_test, tokenizer, model):
    
    x = 0
    pred_labs, tweet_ids = [], []
    while x < X_test.shape[0]:
        #model = DistilBertForSequenceClassification.from_pretrained("models/DistilBert_1", num_labels=4).to(device)
        # y = x + 100
        if((x+10) < X_test.shape[0]):
            y = x + 10
        else:
            y = X_test.shape[0]

        x_test = X_test.iloc[x:y].tolist()
        # t_ids = tweet_id_df.iloc[x:y].tolist()
        # print(x, y, len(x_test))
        inputs = tokenizer(x_test,  padding=True, truncation=True, max_length=500, return_tensors="pt").to(device)
        outputs = model(**inputs)
        
        softmax_labs = list([x for x in outputs[0].softmax(1)])
        softmax_scores = [x.tolist() for x in softmax_labs]
        
        y_pred = [int(x.argmax()) for x in outputs[0].softmax(1)]
        
        pred_labs.extend(y_pred)
        pred_labels.extend(y_pred)
        sm_scores.extend(softmax_scores)
        # tweet_ids.extend(t_ids)
        
        x+=10
        
        if(x%1000 == 0):
            print(x, end = '\r')
        
    return pred_labs, sm_scores

In [18]:
pred_labels, sm_scores = [], []

In [25]:
y_pred, sm_scr = get_predictions(user_resp.clean_text, tokenizer, model)

In [28]:
len(y_pred), len(sm_scr)

(4039474, 4039474)

In [27]:
sys.exit()

SystemExit: 

In [29]:
sm_score_df = pd.DataFrame(sm_scr)

In [30]:
sm_score_df['tweet_id'] = user_resp.tweet_id

In [31]:
sm_score_df.to_csv('dataset/user_resp_MFT_all_softmax.csv')

In [ ]:
[x for x in y_pred[0].softmax(1)][0]

In [ ]:
type(y_pred[0].softmax(1))

In [ ]:
y_pred[0].softmax(1)[0]

In [ ]:
[x.tolist() for x in k]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
user_resp2.shape, user_resp2.columns

In [ ]:
user_resp2.text_x.nunique()

In [ ]:
pd.set_option('display.max_colwidth', None)
user_resp[['text_x', 'clean_text']].iloc[0:100]

In [ ]:
user_resp.to_csv('dataset/MFTC_data.csv')